In [1]:
import cv2

In [2]:
def image_to_feature_vector(image, size=(32, 32)):
    # sliku pretvaramo u sliku fiksnih dimenzija, zapamtite ulaz u CNN uvijek mora biti istih dimenzija
    return cv2.resize(image, size)

In [3]:
import os
print("[INFO] ucitavam slike u listu slika...")
images_paths = os.listdir('podaci/train/')
print("[INFO] ucitao ukupno {} za treniranje i vrednovanje".format(len(images_paths)))

[INFO] ucitavam slike u listu slika...
[INFO] ucitao ukupno 25000 za treniranje i vrednovanje


In [4]:
data = []
labels = []
for (i, image_path) in enumerate(images_paths):
    
    image = cv2.imread('podaci/train/{}'.format(image_path))
    # uzmi iz slike naziv klase buduci da je slika zapisana u obliku oznaka.broj.jpg
    label = image_path.split(os.path.sep)[-1].split(".")[0]
    
    # ucitaj sliku te je smanji na dimenzije 32x32
    features = image_to_feature_vector(image)
    data.append(features)
    labels.append(label)
 
    # prikazi azuriranje svako 1000 slika
    if i > 0 and i % 1000 == 0:
        print("[INFO] obradeno ukupno: {}/{}".format(i, len(images_paths)))

[INFO] obradeno ukupno: 1000/25000
[INFO] obradeno ukupno: 2000/25000
[INFO] obradeno ukupno: 3000/25000
[INFO] obradeno ukupno: 4000/25000
[INFO] obradeno ukupno: 5000/25000
[INFO] obradeno ukupno: 6000/25000
[INFO] obradeno ukupno: 7000/25000
[INFO] obradeno ukupno: 8000/25000
[INFO] obradeno ukupno: 9000/25000
[INFO] obradeno ukupno: 10000/25000
[INFO] obradeno ukupno: 11000/25000
[INFO] obradeno ukupno: 12000/25000
[INFO] obradeno ukupno: 13000/25000
[INFO] obradeno ukupno: 14000/25000
[INFO] obradeno ukupno: 15000/25000
[INFO] obradeno ukupno: 16000/25000
[INFO] obradeno ukupno: 17000/25000
[INFO] obradeno ukupno: 18000/25000
[INFO] obradeno ukupno: 19000/25000
[INFO] obradeno ukupno: 20000/25000
[INFO] obradeno ukupno: 21000/25000
[INFO] obradeno ukupno: 22000/25000
[INFO] obradeno ukupno: 23000/25000
[INFO] obradeno ukupno: 24000/25000


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import numpy as np

le = LabelEncoder ()
labels = le.fit_transform (labels)

data = np.array (data) / 255
labels = np_utils.to_categorical (labels, 2)

print ('[INFO] podjela podataka na skup podataka za treniranje i testiranje')
train_data, test_data, train_labels, test_labels = train_test_split (data, labels, test_size=0.25, random_state=42)

Using TensorFlow backend.


[INFO] podjela podataka na skup podataka za treniranje i testiranje


In [6]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten

def base_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=train_data.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(le.classes_)))
    model.add(Activation('softmax'))
 
    sgd = SGD(lr = 0.1, decay=1e-6, momentum=0.9, nesterov=True)
 
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [7]:
cnn = base_model()
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [9]:
model = cnn.fit (train_data, train_labels, batch_size=32, epochs=20, validation_data=(test_data, test_labels), shuffle=True)

Train on 18750 samples, validate on 6250 samples
Epoch 1/20
18750/18750 [==============================] - 164s - loss: 0.6986 - acc: 0.4971 - val_loss: 0.6994 - val_acc: 0.4974
Epoch 2/20
18750/18750 [==============================] - 210s - loss: 0.6960 - acc: 0.5006 - val_loss: 0.7015 - val_acc: 0.5026
Epoch 3/20
18750/18750 [==============================] - 230s - loss: 0.6959 - acc: 0.4977 - val_loss: 0.6958 - val_acc: 0.5026
Epoch 4/20
18750/18750 [==============================] - 212s - loss: 0.6956 - acc: 0.5049 - val_loss: 0.6931 - val_acc: 0.5026
Epoch 5/20
18750/18750 [==============================] - 214s - loss: 0.6962 - acc: 0.5003 - val_loss: 0.6942 - val_acc: 0.5026
Epoch 6/20
18750/18750 [==============================] - 212s - loss: 0.6961 - acc: 0.5043 - val_loss: 0.7159 - val_acc: 0.4974
Epoch 7/20
18750/18750 [==============================] - 214s - loss: 0.6961 - acc: 0.5023 - val_loss: 0.6964 - val_acc: 0.4974
Epoch 8/20
18750/18750 [========================

In [12]:
model_json = cnn.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

cnn.save_weights("model.h5")
print("Model je spašen na disk...")

Model je spašen na disk...


In [14]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Model učitan s diska")

Model učitan s diska
